In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import os

In [2]:
# Definér transformations for billedet
transform = transforms.Compose([
    transforms.Resize((135, 135)),  # Tilpas til størrelsen på billeder, som mange CNN-modeller forventer
    transforms.ToTensor(),
])

In [3]:
# Definér CNN-arkitekturen
class CNN(nn.Module):
    def __init__(self, num_features):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.fc1 = nn.Linear(64 * 5 * 5, 128) # Efter at have max-poolet 3 gange, så ender vi med billeder der svarer til 5px * 5px
        self.fc2 = nn.Linear(128, num_features)  # Outputlaget har antallet af features som output

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 64 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
# Opret en instans af CNN-modellen
num_features = 12  # Antagelse: Der er 12 features eksklusive Pawpularity
model = CNN(num_features)

In [5]:
# Indlæs den gemte model
model_path = 'trained_model3.pth'
model.load_state_dict(torch.load(model_path))
model.eval()  # Sæt modellen i evalueringsmodus

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=12, bias=True)
)

In [6]:
# Funktion til at forudsige et enkelt billede
def predict_single_image(model, image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Anvend transformationer og tilføj batch-dimension
    with torch.no_grad():
        output = model(image)
        probabilities = torch.sigmoid(output)  # Anvend sigmoid for at få sandsynligheder
    return probabilities.squeeze().cpu().numpy()  # Fjern batch-dimension og konverter til numpy-array


In [7]:
# Indsæt stien til dit billede
image_path = 'pawpularity/train/cbc27493ddaca7127e8cee6705c3b75e.jpg'

In [8]:
# Få sandsynligheder for hver feature
probabilities = predict_single_image(model, image_path)

# Få sikkerhedsprocentsatser for hver feature
feature_names = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
for i, feature_name in enumerate(feature_names):
    print(f'Security percentage for {feature_name}: {probabilities[i] * 100:.2f}%')

FileNotFoundError: [Errno 2] No such file or directory: 'pawpularity/train/cbc27493ddaca7127e8cee6705c3b75e.jpg'